## HRA Hierarchical Tissue Unit Annotation

In this notebook, we will build on [an existing one on hierarchical tissue unit annotation](https://github.com/HickeyLab/Hierarchical-Tissue-Unit-Annotation) by [Dr. John Hickey](https://bme.duke.edu/people/john-hickey/). Outputs from this workbook are featured in Fig. 3 in the paper titled "Human BioMolecular Atlas Program (HuBMAP): 3D Human Reference Atlas Construction and Usage" (accepted for publication in Nature Methods, see preprint [on bioRxiv](https://doi.org/10.1101/2024.03.27.587041)). In this notebook, we will create a Vasculature Common Coordinate Framework (VCCF) distance visualization for the same donor and region (B006, proximal jejunum) as in the paper, see below.

<img src="images/fig_3_original.jpg" width="400">


Concretely, we will take the original CSV file from Dryad with cell positions, types, donor IDs, and extraction sites, and then create a node-dist-vis widget plus a Cell Distance Explorer widget. For more information and documentation on hra-jupyter-widgets, please see [https://github.com/x-atlas-consortia/hra-jupyter-widgets/blob/main/usage.ipynb](https://github.com/x-atlas-consortia/hra-jupyter-widgets/blob/main/usage.ipynb).

## Load libraries

In [1]:
# Import native packages
import os
from pprint import pprint
import csv

In [2]:
#Install and import external packages
%pip install matplotlib pandas ipywidgets hra_jupyter_widgets

import matplotlib.pyplot as plt
import pandas as pd
import ipywidgets as widgets

# Import hra-jupyter-widgets. For documentation, please see https://github.com/x-atlas-consortia/hra-jupyter-widgets/blob/main/usage.ipynb
from hra_jupyter_widgets import CdeVisualization

Note: you may need to restart the kernel to use updated packages.


## Download data from Dryad

Let's get the data from Dryad. Note that this is a ~3GB file. We will use `curl` to dowload it.

In [3]:
# Make sure the data folder is present
folder_path = "data/"

if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print(f"Folder '{folder_path}' created.")
else:
    print(f"Folder '{folder_path}' already exists.")

# Define the path to the file. 
file_path = f'{folder_path}/23_09_CODEX_HuBMAP_alldata_Dryad_merged.csv'

# Check if the file exists
if not os.path.exists(file_path):
    # If the file doesn't exist, run the curl command
    !curl -L https://datadryad.org/api/v2/files/2572152/download -o {file_path}
    print(f"File downloaded and saved at {file_path}")
else:
    print(f"File already exists at {file_path}")

Folder 'data/' already exists.
File already exists at data//23_09_CODEX_HuBMAP_alldata_Dryad_merged.csv


## Read data as DataFrame

In [4]:
# Read the CSV file and convert it to a df
df = pd.read_csv(file_path, index_col=0)

In [5]:
# Only keep cells from one dataset by selecting 1 donor and 1 region
df_filtered = df[(df['donor'] == "B012") & (
    df['unique_region'] == "B012_Proximal jejunum")]

In [6]:
df_filtered['Cell Type'] = df_filtered['Cell Type'].astype(str) + '_Cell Type'
df_filtered['Neighborhood'] = df_filtered['Neighborhood'].astype(str) + '_Neighborhood'
df_filtered['Community'] = df_filtered['Community'].astype(str) + '_Community'
df_filtered['Tissue Unit'] = df_filtered['Tissue Unit'].astype(str) + '_Tissue Unit'

C:\Users\abueckle\AppData\Local\Temp\1\ipykernel_36396\1316168077.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Cell Type'] = df_filtered['Cell Type'].astype(str) + '_Cell Type'
C:\Users\abueckle\AppData\Local\Temp\1\ipykernel_36396\1316168077.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Neighborhood'] = df_filtered['Neighborhood'].astype(str) + '_Neighborhood'
C:\Users\abueckle\AppData\Local\Temp\1\ipykernel_36396\1316168077.py:3: SettingWithCopyWarning: 
A value is t

In [7]:
# Next, let's define a function that turns a DataFrame into a node list that can then be passed into the CdeVisualization or NodeDistVis widget
def make_node_list(df:pd.DataFrame, is_3d:bool = False):
  """Turn a DataFrame into a list of dicts for passing them into a HRA widget

  Args:
      df (pd.DataFrame): A DataFrame with cells
  """
  
  # If the df does not have a z-axis column, let's add one and set all cells to 0
  if not is_3d:
    df.loc[:, ('z')] = 0
  
  node_list = [{'x': row['x'], 'y': row['y'], 'z': row['z'], 'Cell Type': row['combined_info']}
                 for index, row in df.iterrows()]

  return node_list
  

## Next, let's make four regions and visualize a 3D tissue stack.

To showcase the 3D rendering capabilities of our `CDEVisualization` widget, we will display the same section as we showed with the `NodeDistVis` widget above, but we will show it three times with an added offset along the z-axis.

In [8]:
# Create a new data frame with values from the NodeDistVis example
df = df_filtered

# indicate the number of layers you would like to show. In our case, let's show 3.
n_layers = 4   # One layer for cell type, one for neighborhood, one for community, one for tissue unit

# Create a list to hold all the data frames
df_list = [df]

for i in range(0, n_layers-1):

  # Create a copy of this new DataFrame
  df_copy = df.copy()

  # Modify a column in the copied rows (e.g., change values in column 'B')
  df_copy['unique_region'] = f'copy_{i}'  

  # Add df_copy to list of df
  df_list.append(df_copy)
  
# Concatenate the original DataFrame with the modified copies
df_combined = pd.concat(df_list, ignore_index=True)

df_filtered_3d = df_combined

In [9]:
# Set a z-offset
offset = 2500

# Set z axis (or any other axis) by region
df_filtered_3d['z'] = df_filtered_3d['unique_region'].apply(lambda v: 0 if v == 'B012_Proximal jejunum'
                                                            else offset if v == 'copy_0'
                                                            else offset * 2 if v == 'copy_1'
                                                            else offset * 3
                                                            )

# Make new df with only x, y, z, Cell Type, and Neighborhood columns
df_cells_3d = df_filtered_3d[['x', 'y', 'z', 'Cell Type', 'Neighborhood', 'Community', 'Tissue Unit']]

cell_type_z0 = df_cells_3d[df_cells_3d['z'] == 0]['Cell Type']
neighborhood_z1 = df_cells_3d[df_cells_3d['z'] == offset]['Neighborhood']
community_z2 = df_cells_3d[df_cells_3d['z'] == offset *2]['Community']
tissueunit_z3 = df_cells_3d[df_cells_3d['z'] == offset *3]['Tissue Unit']

combined_info = pd.concat([cell_type_z0, neighborhood_z1, community_z2, tissueunit_z3])

df_cells_3d['combined_info'] = combined_info

df_cells_3d_filtered = df_cells_3d[['x', 'y', 'z', 'combined_info']]

C:\Users\abueckle\AppData\Local\Temp\1\ipykernel_36396\864382709.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cells_3d['combined_info'] = combined_info


# Display

In [10]:
node_list = make_node_list(df_cells_3d_filtered, True)
edge_list = [["Cell ID","Target ID", "X1", "Y1", "Z1", "X2", "Y2", "Z2"]] # this makes an empty edges list

In [11]:
# Finally, let's instantiate the CDEVisualization class with our node_list as parameter.
cde = CdeVisualization(
    nodes=node_list,
    node_target_key = "Cell Type",
    edges=edge_list,
)

# Display our new widget
display(cde)

CdeVisualization(age=None, color_map=None, color_map_key=None, color_map_value_key=None, creation_timestamp=No…

## Cull amount of data exported

In [13]:
cull_data = False #cull data?
n = 10 #keep every nth row

if cull_data:
  df_cells_3d_filtered = df_cells_3d[['x', 'y', 'z', 'combined_info']]
  n = 10
  df_nth = df_cells_3d_filtered.iloc[::n]
  df_cells_3d_filtered = df_nth
  print(f"df_cells_3d_filtered has length {len(df_cells_3d_filtered)}.")
  print(df_cells_3d_filtered)

# Prepare data for export and use in HRA Organ Gallery

In [14]:
df_cells_3d_filtered

,x,y,z,combined_info
0,7109.0,7160.0,0,Cycling TA_Cell Type
1,4803.0,55.0,0,Cycling TA_Cell Type
2,7203.0,7413.0,0,Cycling TA_Cell Type
3,424.0,1437.0,0,Cycling TA_Cell Type
4,93.0,344.0,0,Cycling TA_Cell Type
...,...,...,...,...
130075,7604.0,6824.0,7500,Mucosa_Tissue Unit
130076,664.0,4597.0,7500,Muscularis externa_Tissue Unit
130077,5695.0,4301.0,7500,Mucosa_Tissue Unit
130078,61.0,3230.0,7500,Muscularis externa_Tissue Unit


In [15]:
n = 10  # keep every nth row

# separate into 4 dfs
df_list = []

# get dfs
levels = ['Cell Type', 'Neighborhood', 'Community', 'Tissue Unit']

df_cells_3d_filtered

for l in levels:
  df_list.append(df_cells_3d_filtered[df_cells_3d_filtered['combined_info'].str.contains(l, case=False, na=False)][::n])

# export all
counter = 0
for df in df_list:
  appendix = ""
  last_column_name = df.columns[-1]  # Get the last column name
  # print(df_list.index(df))
  file_name = f"df_{levels[counter]}.csv"  # Create a dynamic file name
  df.to_csv(f"data/output/{file_name}", index=False)  # Export the DataFrame to CSV
  print(f"Exported {file_name} with {len(df)} rows")
  counter = counter + 1 

Exported df_Cell Type.csv with 3252 rows
Exported df_Neighborhood.csv with 3252 rows
Exported df_Community.csv with 3252 rows
Exported df_Tissue Unit.csv with 3252 rows
